In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('dataset/movies.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies.shape

(62423, 3)

In [5]:
import re

def clean_title(title):
    return re.sub('[^a-zA-Z0-9 ]', '', title.lower())

In [6]:
movies['clean_title'] = movies['title'].apply(clean_title)

In [7]:
movies.head()

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii 1995


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [10]:
tfidf = vectorizer.fit_transform(movies['clean_title'])

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [12]:
def search(title):
    title = clean_title(title)
    vec_title = vectorizer.transform([title])
    similarity = cosine_similarity(vec_title, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices][::-1]
    return results

In [13]:
search('Thor')

,movieId,title,genres,clean_title
25069,122916,Thor: Ragnarok (2017),Action|Adventure|Sci-Fi,thor ragnarok 2017
40293,159089,I Am Thor (2015),Documentary,i am thor 2015
16312,86332,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX,thor 2011
20513,106072,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX,thor the dark world 2013
43265,165639,While You Were Fighting: A Thor Mockumentary (...,Comedy|Fantasy|Sci-Fi,while you were fighting a thor mockumentary 2016


In [14]:
import ipywidgets as wiz
from IPython.display import display

In [15]:
movie_input = wiz.Text(value = 'Toy Story', description = 'Movie Title:', disabled = False)

In [16]:
movie_list = wiz.Output()

In [17]:
def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data['new']
        if len(title) > 5:
            display(search(title))

In [18]:
movie_input.observe(on_type, names='value')

In [19]:
display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [20]:
ratings = pd.read_csv('dataset/ratings.csv')

In [21]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [22]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [23]:
ratings.shape

(25000095, 4)

In [24]:
# cardrequest@bracbank.com

In [25]:
movie_id = 1

In [26]:
similar_users = ratings[(ratings['movieId'] == movie_id) & (ratings['rating'] > 4)]['userId'].unique()

In [27]:
similar_users

array([    36,     75,     86, ..., 162527, 162530, 162533])

In [28]:
similar_user_recs = ratings[(ratings['userId'].isin(similar_users)) & (ratings['rating'] > 4)]['movieId']

In [29]:
similar_user_recs

5101            1
5105           34
5111          110
5114          150
5127          260
            ...  
24998854    60069
24998861    67997
24998876    78499
24998884    81591
24998888    88129
Name: movieId, Length: 1358326, dtype: int64

In [30]:
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

similar_user_recs = similar_user_recs[similar_user_recs > .1]

In [31]:
similar_user_recs

1        1.000000
318      0.445607
260      0.403770
356      0.370215
296      0.367295
           ...   
953      0.103053
551      0.101195
1222     0.100876
745      0.100345
48780    0.100186
Name: movieId, Length: 113, dtype: float64

In [32]:
all_users = ratings[(ratings['movieId'].isin(similar_user_recs.index)) & (ratings['rating'] > 4)]

In [33]:
all_user_recs = all_users['movieId'].value_counts() / len(all_users['userId'].unique())

In [34]:
all_user_recs

318      0.342220
296      0.284674
2571     0.244033
356      0.235266
593      0.225909
           ...   
551      0.040918
50872    0.039111
745      0.037031
78499    0.035131
2355     0.025091
Name: movieId, Length: 113, dtype: float64

In [35]:
recs_percentage = pd.concat([similar_user_recs, all_user_recs], axis=1)
recs_percentage.columns = ['similar', 'all']

In [36]:
recs_percentage

,similar,all
1,1.000000,0.124728
318,0.445607,0.342220
260,0.403770,0.222207
356,0.370215,0.235266
296,0.367295,0.284674
...,...,...
953,0.103053,0.045792
551,0.101195,0.040918
1222,0.100876,0.066877
745,0.100345,0.037031


In [37]:
recs_percentage['score'] = recs_percentage['similar'] / recs_percentage['all']

In [38]:
recs_percentage = recs_percentage.sort_values('score', ascending=False)

In [39]:
recs_percentage

,similar,all,score
1,1.000000,0.124728,8.017414
3114,0.280648,0.053706,5.225654
2355,0.110539,0.025091,4.405452
78499,0.152960,0.035131,4.354038
4886,0.235147,0.070811,3.320783
...,...,...,...
2858,0.216724,0.167634,1.292845
296,0.367295,0.284674,1.290232
79132,0.166817,0.131384,1.269693
4973,0.142501,0.112405,1.267747


In [40]:
recs_percentage.head(10).merge(movies, left_index=True, right_on='movieId')

,similar,all,score,movieId,title,genres,clean_title
0,1.000000,0.124728,8.017414,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995
3021,0.280648,0.053706,5.225654,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,toy story 2 1999
2264,0.110539,0.025091,4.405452,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy,bugs life a 1998
14813,0.152960,0.035131,4.354038,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX,toy story 3 2010
4780,0.235147,0.070811,3.320783,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,monsters inc 2001
580,0.216618,0.067513,3.208539,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,aladdin 1992
6258,0.228139,0.072268,3.156862,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,finding nemo 2003
587,0.179400,0.059977,2.991150,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,beauty and the beast 1991
8246,0.203504,0.068453,2.972889,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,incredibles the 2004
359,0.253411,0.085764,2.954762,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,lion king the 1994


In [41]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings['movieId'] == movie_id) & (ratings['rating'] > 4)]['userId'].unique()
    similar_user_recs = ratings[(ratings['userId'].isin(similar_users)) & (ratings['rating'] > 4)]['movieId']
    
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    
    all_users = ratings[(ratings['movieId'].isin(similar_user_recs.index)) & (ratings['rating'] > 4)]
    all_user_recs = all_users['movieId'].value_counts() / len(all_users['userId'].unique())
    
    recs_percentage = pd.concat([similar_user_recs, all_user_recs], axis=1)
    recs_percentage.columns = ['similar', 'all']
    
    recs_percentage['score'] = recs_percentage['similar'] / recs_percentage['all']
    recs_percentage = recs_percentage.sort_values('score', ascending=False)
    
    return recs_percentage.head(10).merge(movies, left_index=True, right_on='movieId')[['score', 'title', 'genres']]

In [42]:
movie_name_input = wiz.Text(value = 'Toy Story', description = 'Movie Title:', disabled = False)

recommendation_list = wiz.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data['new']
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]['movieId']
            display(find_similar_movies(movie_id))
            
movie_name_input.observe(on_type, names='value')
display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()